# Libraries

In [1]:
import sys
import os
import scanpy as sc
import pandas as pd
import numpy as np

# Configurations

In [22]:
project_dir = "..\\"
type_ = 'human_brain' #all files name will be depend on this 
species='human'
tissue='brain'

In [3]:
meta = pd.read_csv('meta_all_gene.csv')
# filter the meta file to get only the human brain 
meta = meta[(meta['species'] == species) & (meta['tissue'] == tissue)]
# path = '.'


In [4]:
meta

,slide,species,tissue,pmid,title,abstract,keywords,involve_cancer,tech,spot_num,gene_num
226,GSE179572_GSM5420749,human,brain,"35584630,35707680",Title 1: Distinct phenotypic states and spatia...,Abstract 1: Metastatic disease in the brain is...,Keywords 1: CD8(+) T cells; TCR-sequencing; br...,True,Visium,2112,36601
227,GSE179572_GSM5420750,human,brain,"35584630,35707680",Title 1: Distinct phenotypic states and spatia...,Abstract 1: Metastatic disease in the brain is...,Keywords 1: CD8(+) T cells; TCR-sequencing; br...,True,Visium,1716,36601
228,GSE179572_GSM5420751,human,brain,"35584630,35707680",Title 1: Distinct phenotypic states and spatia...,Abstract 1: Metastatic disease in the brain is...,Keywords 1: CD8(+) T cells; TCR-sequencing; br...,True,Visium,946,36601
229,GSE179572_GSM5420752,human,brain,"35584630,35707680",Title 1: Distinct phenotypic states and spatia...,Abstract 1: Metastatic disease in the brain is...,Keywords 1: CD8(+) T cells; TCR-sequencing; br...,True,Visium,1298,36601
230,GSE179572_GSM5420753,human,brain,"35584630,35707680",Title 1: Distinct phenotypic states and spatia...,Abstract 1: Metastatic disease in the brain is...,Keywords 1: CD8(+) T cells; TCR-sequencing; br...,True,Visium,4342,36601
...,...,...,...,...,...,...,...,...,...,...,...
1046,Human_Brain_Maynard_02082021_Visium_151672,human,brain,33558695,Transcriptome-scale spatial gene expression in...,We used the 10x Genomics Visium platform to de...,NaN,False,Visium,4015,33538
1047,Human_Brain_Maynard_02082021_Visium_151673,human,brain,33558695,Transcriptome-scale spatial gene expression in...,We used the 10x Genomics Visium platform to de...,NaN,False,Visium,3639,33538
1048,Human_Brain_Maynard_02082021_Visium_151674,human,brain,33558695,Transcriptome-scale spatial gene expression in...,We used the 10x Genomics Visium platform to de...,NaN,False,Visium,3673,33538
1049,Human_Brain_Maynard_02082021_Visium_151675,human,brain,33558695,Transcriptome-scale spatial gene expression in...,We used the 10x Genomics Visium platform to de...,NaN,False,Visium,3592,33538


In [25]:
def find_hvg(slide, tech):
    try:
        # Construct the correct file path
        file_path = os.path.join(project_dir, "data", tech, "gene_exp", f"{slide}_count.csv")

        # Check if the file exists before loading
        if os.path.exists(file_path):
            gene_exp_slide = pd.read_csv(file_path, sep=',', index_col=0)
            print("File loaded successfully!")
        else:
            print(f"Error: File not found at {file_path}")
            return  # Exit function if file is not found

        # Process the data
        adata = sc.AnnData(gene_exp_slide)
        adata.var_names_make_unique()
        sc.pp.filter_cells(adata, min_genes=1)
        sc.experimental.pp.highly_variable_genes(adata, n_top_genes=128)
        
        # Sort genes by highly_variable
        hvg_list = adata.var['highly_variable_rank']
        hvg_list = hvg_list.sort_values().dropna()

        # Subset data to HVG genes
        adata_hvg = adata[:, hvg_list.index]
        sc.pp.normalize_total(adata_hvg)
        sc.pp.log1p(adata_hvg)

        # Convert to DataFrame
        hvg = pd.DataFrame(adata_hvg.X, index=adata_hvg.obs.index)

        # Ensure the HVG directory exists
        hvg_dir = os.path.join(project_dir, "data", "HVG")
        os.makedirs(hvg_dir, exist_ok=True)

        # Save HVG file
        hvg_path = os.path.join(hvg_dir, f"{slide}_count_hvg.csv")
        hvg.to_csv(hvg_path, sep=',')
        print(f"HVG file saved: {hvg_path}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [6]:
# def find_hvg(slide, tech):
#     try:
#         gene_exp_slide = pd.read_csv(f'{path}/{tech}/gene_exp/{slide}.csv',sep=',',index_col=0)
#         adata = sc.AnnData(gene_exp_slide)
#         adata.var_names_make_unique()
        
#         sc.pp.filter_cells(adata, min_genes=1)
#         sc.experimental.pp.highly_variable_genes(adata, n_top_genes=128)
        
#         # sort genes by highly_variable
#         adata.var_names_make_unique()
#         hvg_list = adata.var['highly_variable_rank']
#         hvg_list = hvg_list.sort_values()
#         hvg_list = hvg_list.dropna()
        
#         adata_hvg = adata[:, hvg_list.index]
#         sc.pp.normalize_total(adata_hvg)
#         sc.pp.log1p(adata_hvg)
#         hvg = adata_hvg.X
#         hvg = pd.DataFrame(hvg)
#         hvg.index = adata_hvg.obs.index
#         hvg.to_csv(f'./HVG/{slide}_count_hvg.csv',sep=',')
#         print(f"FOUND HVG: {path}/{tech}/gene_exp/{slide}.csv")
#     except:
#         pass

# Find HVGs

In [27]:
# Process each row efficiently
for row in meta.itertuples(index=False):
    slide, tech = row.slide, row.tech  # Access fields by name
    # Check for missing values
    if pd.notna(slide) and pd.notna(tech):
        find_hvg(slide, tech)
    else:
        print(f"Skipping row due to missing values: slide={slide}, tech={tech}")

File loaded successfully!


d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\scanpy\preprocessing\_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


HVG file saved: ..\data\HVG\GSE179572_GSM5420749_count_hvg.csv
File loaded successfully!


d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\scanpy\preprocessing\_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


HVG file saved: ..\data\HVG\GSE179572_GSM5420750_count_hvg.csv
File loaded successfully!


...

HVG file saved: ..\data\HVG\GSE179572_GSM5420752_count_hvg.csv
File loaded successfully!


d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\scanpy\experimental\pp\_highly_variable_genes.py:154: UserWarning: `flavor='pearson_residuals'` expects raw count data, but non-integers were found.
  warnings.warn(
d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\scanpy\preprocessing\_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\scanpy\preprocessing\_normalization.py:235: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


HVG file saved: ..\data\HVG\GSE179572_GSM5420753_count_hvg.csv
File loaded successfully!


d:\DEPI\FinalProject\TissueInsight\venv\Lib\site-packages\scanpy\preprocessing\_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


HVG file saved: ..\data\HVG\GSE179572_GSM5420754_count_hvg.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591748_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591749_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591750_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591751_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591752_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591753_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591754_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE184510_GSM5591755_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE232910_GSM7392324_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE232910_GSM7392325_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE232910_GSM7392326_count.csv
Error: File not found at ..\data\Visium\gene_exp\GSE23291

In [8]:
# for i in range(len(meta)):
#     slide = meta['slide'][i]
#     tech = meta['tech'][i]
#     find_hvg(slide, tech)

# Combine HVG of Human brain

In [9]:
# data = meta.loc[(meta['species'] == 'human') & (meta['tissue'] == 'skin'),:]
# data.slide.to_csv(f'human_skin_slide.csv',index=False,header=False)

In [ ]:
# import os

# Define file path
slide_dir = os.path.join("../data/overlap-hvg")
os.makedirs(slide_dir, exist_ok=True)

# Filter data for human skin samples
# data = meta.loc[(meta['species'] == 'human') & (meta['tissue'] == 'brain'),:]

# Ensure the column 'slide' exists before saving

if 'slide' in meta.columns:
    meta['slide'].to_csv(f"{slide_dir}/{type_}_slide.csv", index=False, header=False)
    print(f"Slide file created: {slide_dir}")
else:
    print("Error: 'slide' column not found in meta data!")


Slide file created: ../data/overlap-hvg


In [ ]:
# import os
# import pandas as pd

path = '../data'
combined_hvg_dir='../data/combined_hvg/'
output_file = f'{combined_hvg_dir}{type_}_count_hvg.csv'
slide_file = f'../data/overlap-hvg/{type_}_slide.csv'

# Remove the output file if it exists
os.makedirs(combined_hvg_dir, exist_ok=True)

if os.path.exists(output_file):
    os.remove(output_file)
# Check if slide file exists
    if not os.path.exists(slide_file):
        print(f"Error: Slide file not found at {slide_file}")
    else:
        # Read slide names from file
        with open(slide_file, 'r') as f:
            slides = f.read().splitlines()

        # Process each slide file
        for slide in slides:
            input_file = os.path.join(path, 'HVG', f'{slide}_count_hvg.csv')
            if os.path.exists(input_file):
                df = pd.read_csv(input_file, skiprows=1)  # Skip the first row (header)
                df.to_csv(output_file, mode='a', index=False, header=False)
            print(slide)


GSE179572_GSM5420749
GSE179572_GSM5420750
GSE179572_GSM5420751
GSE179572_GSM5420752
GSE179572_GSM5420753
GSE179572_GSM5420754
GSE184510_GSM5591748
GSE184510_GSM5591749
GSE184510_GSM5591750
GSE184510_GSM5591751
GSE184510_GSM5591752
GSE184510_GSM5591753
GSE184510_GSM5591754
GSE184510_GSM5591755
GSE232910_GSM7392324
GSE232910_GSM7392325
GSE232910_GSM7392326
GSE232910_GSM7392327
GSE232910_GSM7392328
GSE235672_GSM7507311
GSE235672_GSM7507312
GSE235672_GSM7507313
GSE235672_GSM7507314
GSE235672_GSM7507315
GSE235672_GSM7507316
GSE235672_GSM7507317
GSE235672_GSM7507318
GSE235672_GSM7507319
GSE235672_GSM7507320
GSE235672_GSM7507321
GSE235672_GSM7507322
GSE235672_GSM7507323
GSE235672_GSM7507324
GSE235672_GSM7507325
GSE235672_GSM7507326
Human_Brain_10X_02132023_Visium
Human_Brain_10X_10272020_Visium_Cerebellum_WholeTranscriptome
Human_Brain_10X_10272020_Visium_WholeTranscriptome
Human_Brain_Kwon_10162023_Visium_V10A27004_A1_Br3874
Human_Brain_Kwon_10162023_Visium_V10A27004_D1_Br3880
Human_Brain_Kw

# Overlap HVG

In [12]:
# data = meta.loc[(meta['species'] == 'human') & (meta['tissue'] == 'skin'),:]
meta.index = range(len(meta.index))
gene_name_overlap = []

In [13]:
# here we find the overlap genes in human brain


for i in range(len(meta.index)):
    try:
        gene_exp_slide = pd.read_csv(f'../data/{meta.tech[i]}/gene_exp/{meta.slide[i]}.csv',sep=',',nrows=1,index_col=0)
        if i == 12:
            gene_name_overlap = gene_exp_slide.columns
        else:
            gene_name_overlap = gene_name_overlap.intersection(gene_exp_slide.columns)
        print(i)
    except:
        pass

pd.DataFrame(gene_name_overlap).to_csv(f'{type_}_gene.csv',index=False,header=False)

In [14]:
gene_list = type_+'_gene'


In [15]:
if tissue == 'all':
    data = meta.loc[(meta['species'] == species), :].reset_index(drop=True)
    data.slide.to_csv(f'{species}_slide.csv', index=False, header=False)
else:
    data = meta.loc[(meta['species'] == species) & (meta['tissue'] == tissue), :].reset_index(drop=True)
    data.slide.to_csv(f'{species}_{tissue}_slide.csv', index=False, header=False)

In [16]:
data[data['slide'] == 'GSE144239_GSM4565823']

,slide,species,tissue,pmid,title,abstract,keywords,involve_cancer,tech,spot_num,gene_num


In [17]:
for index in range(len(data.index)):
    try:
        slide = data['slide'][index]
        gene_exp_slide = pd.read_csv(f'./{data.tech[index]}/gene_exp/{slide}.csv',sep=',',index_col=0)
        overlap_gene = pd.read_csv(f'{gene_list}.csv',header=None)
        gene_exp_slide = gene_exp_slide.loc[:,overlap_gene[0]]
        gene_exp_slide.to_csv(f'./overlap-hvg/{data.slide[index]}_{gene_list}.csv',sep=',')
        print(f"DONE: ./{data.tech[index]}/gene_exp/{data.slide[index]}.csv")
    except:
        pass

In [24]:
import os
import pandas as pd

# Define variables
path = '../data/overlap-hvg/'
output_file = f'{type_}_count_overlap.csv'
slide_file = f'{type_}_slide.csv'

# Remove existing output file if it exists
if os.path.exists(output_file):
    os.remove(output_file)

# Read slide names
with open(slide_file, 'r') as file:
    slides = file.read().splitlines()

# Process slides
missing_slides = []
header_written = False

for slide in slides:
    input_file = os.path.join(path, f"{slide}_{type_}_gene.csv")

    if os.path.isfile(input_file):
        df = pd.read_csv(input_file, skiprows=1)  # Skip first row (header)

        # Write header only for the first file
        if not header_written:
            df.to_csv(output_file, mode='w', index=False, header=True)
            header_written = True
        else:
            df.to_csv(output_file, mode='a', index=False, header=False)  # Append without header
    else:
        missing_slides.append(slide)

# Ensure output file exists (even if no slides were found)
if not os.path.exists(output_file):
    with open(output_file, 'w') as f:
        f.write("")  # Create an empty file

# Print missing slides
if missing_slides:
    print("Missing slides:", ", ".join(missing_slides))
else:
    print("All slides processed successfully.")


Missing slides: GSE179572_GSM5420749, GSE179572_GSM5420750, GSE179572_GSM5420751, GSE179572_GSM5420752, GSE179572_GSM5420753, GSE179572_GSM5420754, GSE184510_GSM5591748, GSE184510_GSM5591749, GSE184510_GSM5591750, GSE184510_GSM5591751, GSE184510_GSM5591752, GSE184510_GSM5591753, GSE184510_GSM5591754, GSE184510_GSM5591755, GSE232910_GSM7392324, GSE232910_GSM7392325, GSE232910_GSM7392326, GSE232910_GSM7392327, GSE232910_GSM7392328, GSE235672_GSM7507311, GSE235672_GSM7507312, GSE235672_GSM7507313, GSE235672_GSM7507314, GSE235672_GSM7507315, GSE235672_GSM7507316, GSE235672_GSM7507317, GSE235672_GSM7507318, GSE235672_GSM7507319, GSE235672_GSM7507320, GSE235672_GSM7507321, GSE235672_GSM7507322, GSE235672_GSM7507323, GSE235672_GSM7507324, GSE235672_GSM7507325, GSE235672_GSM7507326, Human_Brain_10X_02132023_Visium, Human_Brain_10X_10272020_Visium_Cerebellum_WholeTranscriptome, Human_Brain_10X_10272020_Visium_WholeTranscriptome, Human_Brain_Kwon_10162023_Visium_V10A27004_A1_Br3874, Human_Brain_

In [ ]:
def overlap_hvg(tissue_type):
    # Here we take input of combined gene expression of one tissue type
    # see below for how to generate this file
    gene_exp_slide = pd.read_csv(f'{project_dir}/{tissue_type}_count_overlap.csv',sep=',',index_col=0,header=None)
    gene_exp_slide.columns = ['gene'+str(i) for i in range(gene_exp_slide.shape[1])]
    adata = sc.AnnData(gene_exp_slide)
    sc.pp.filter_cells(adata, min_genes=1)
    sc.pp.filter_genes(adata, min_cells=1)
    # Normalizing to median total counts
    sc.pp.normalize_total(adata)
    # Logarithmize the data
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes=100)
    # sort genes by highly_variable
    adata_hvg = adata[:, adata.var.highly_variable]
    hvg = adata_hvg.X
    hvg = pd.DataFrame(hvg)
    hvg.index = adata_hvg.obs.index
    hvg.to_csv(f'./all/{tissue_type}_count_overlap_hvg.csv',sep=',')

overlap_hvg("human_skin")